In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import psycopg2

In [2]:
URL = 'https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=4897&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
link_list = []
p = 0

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36')
options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(0.05)

while len(link_list) < 50:
    time.sleep(0.05)
    html = driver.find_element(By.TAG_NAME, "html")
    time.sleep(0.05)

    for k in range(400):
        html.send_keys(Keys.DOWN)

    offers = driver.find_elements(By.CLASS_NAME, '_93444fe79c--link--VtWj6')

    for elem in offers:
        link_list.append(elem.get_attribute('href'))

    p += 1
    next_page = f'https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={p+1}&region=4897&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
    driver.get(next_page)

In [3]:
flat_list = []
driver = webdriver.Chrome(options=options)
id = 0

for link in link_list:
    driver.get(link)
    flat = {}
    address_str = ''

    flat['id'] = id

    descript = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/div[3]')
    elements = descript.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--Jp5Qv')
    price = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--amount--ON6i1')
    address = driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--address--SMU25')

    try:
        metro = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--undergrounds--sGE99')
    except:
        pass

    for elem in address:
        address_str += elem.text

    for item in elements:
        item = item.text.split('\n')
        flat[item[0]] = item[1]

    flat['price'] = price.text
    flat['metro'] = metro.text
    flat['address'] = address_str
    flat_list.append(flat)
    id += 1

In [4]:
# data preparation

columns = set(('id', 'Общая площадь', 'Этаж', 'Год сдачи', 'Дом', 'Отделка', 'price', 'metro', 'address', 'Жилая площадь', 'Площадь кухни'))

for flat in flat_list:
    for key in columns - set(flat.keys()):
        flat[key] = 'None' 

    if flat['metro'] == '':
        flat['metro'] = 'None'
    if 'Год постройки' in set(flat.keys()): del flat['Год постройки']

In [6]:
flat_lst_copy = flat_list.copy()

In [ ]:
def price_corr(price)

In [27]:

conn = psycopg2.connect(host='localhost', database='flats', user='postgres', password='2109')
cur = conn.cursor()


In [40]:
columns = set(('id', 'Общая площадь', 'Этаж', 'Год сдачи', 'Дом', 'Отделка', 'price', 'metro', 'address', 'Жилая площадь', 'Площадь кухни'))

for item in flat_list:
    if item['metro'] == '':
        item['metro'] = 'None'
    if 'Год постройки' in set(item.keys()): del item['Год постройки']

In [55]:

with conn.cursor() as curs:
    for item in flat_list:
        tuple_insert = tuple((item['id'], item['Общая площадь'], item['Этаж'], item['Год сдачи'], item['Дом'], item['Отделка'],\
                              item['price'], item['metro'], item['address'], item['Площадь кухни'], item['Жилая площадь']))
        curs.execute("INSERT INTO flats (id, total_square, floor, year, rented, decor, price, metro, address, living_sq, kitchen_sq) VALUES (%s, %s, %s, \
                    %s, %s, %s, %s, %s, %s, %s, %s)", tuple_insert)
        conn.commit()


In [58]:
conn.rollback()

In [59]:
cur.execute("SELECT * FROM flats")
rows = cur.fetchall()
